In [229]:
import pandas as pd
import numpy as np

df = pd.read_csv('pre_processed.csv')
df.Number = df.Number.astype('Int64')



In [230]:
orig = pd.read_csv('pre_processed.csv')


In [231]:

def fill_deck_and_side_from_cabin(df):
    # Define a mask to identify rows where 'Number' is NaN and 'Cabin' is not NaN
    mask = df['Number'].isna() & df['Cabin'].notna()
    
    # Use the mask to update only the filtered rows, converting types appropriately
    df.loc[mask, ['Deck', 'Number', 'Side']] = df.loc[mask, 'Cabin'].apply(
        lambda x: pd.Series({
            'Deck': x.split('/')[0],
            'Number': int(x.split('/')[1]),  # Explicit conversion to integer here
            'Side': x.split('/')[2]
        })
    )

    return df

In [232]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

homeplanets = ['Earth', 'Europa', 'Mars']

all_cabin_sides = ['P','S']

all_cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [233]:
def multiple_decks_in_group(df,row):
    return len(df[df.Group == row.Group].dropna(subset ='Deck').Deck.unique()) > 1
        

In [234]:
def fill_potential_decks(df):
    
    def func_potential_decks(row):
        if pd.isna(row.Cabin):
            if row.Bills == 0 and not pd.isna(row.HomePlanet):
                if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
                    return decks_by_planet_no_bills[row.HomePlanet]
                
            if not pd.isna(row.HomePlanet):
                return decks_by_planet[row.HomePlanet]
            
            else:
                return all_cabin_decks
            
    df['potential_decks'] = df.apply(func_potential_decks, axis = 1)
    return df

In [235]:
def fill_potential_sides(df):
    
    def func_potential_sides(row):
        if pd.isna(row.Cabin):
            if row.GroupSize > 1:
                group = df[df.Group == row.Group].dropna(subset = 'Side')
                if len(group) > 0:
                    return [group.iloc[0].Side]
            return ['P','S']
        
    df['potential_sides'] = df.apply(func_potential_sides,axis = 1)
    return df

In [283]:
df = fill_potential_decks(df)
df = fill_potential_sides(df)

# Filling missed Cabins


#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

Finding groups that have more than 1 member that all share the same cabin and that dont have other nans in group
* make sure that there aren't other nans in group or else it might change results if there arent potentially other cabins it could go to if the other wasnt nan *

## empty room with only one comptabile


In [284]:

                    
def rooms_to_fill(df):
    rooms = {}
    for deck in all_cabin_decks:
        rooms[deck] = {'P': [], 'S': []} 
        for side in all_cabin_sides:
            rooms_seen = df[(df.Deck == deck) & (df.Side == side) & (df.Number.notna())].Number.astype(int).tolist()
            largest_room_number = max(rooms_seen, default=-1) 

            for i in range(largest_room_number + 1):
                if i not in rooms_seen:
                    rooms[deck][side].append(f"{deck}/{i}/{side}")

    return rooms
           

In [285]:

 
def row_is_compatible_with_cabin(index, row, deck, side, room_number_to_fill):
        if deck not in row.potential_decks or side not in row.potential_sides:
            return False
        
        before = df.loc[(df.index < index) & (df.Deck == deck) & (df.Side == side), 'Number'].dropna().unique()
        after = df.loc[(df.index > index) & (df.Deck == deck) & (df.Side == side), 'Number'].dropna().unique()
        
        if len(before) > 0 and len(after) > 0:
            return max(before) < room_number_to_fill < min(after)
        return False
    
                        
def empty_room_one_compatible(df):
    
    empty_cabins = rooms_to_fill(df)

    for deck, sides in empty_cabins.items():
        for side, cabins in sides.items():
            for cabin in cabins:
                room_number_to_fill = int(cabin.split('/')[1])
                compatible_indices = [
                    index for index, row in df[df.Cabin.isna()].iterrows()
                    if row_is_compatible_with_cabin(index, row, deck, side, room_number_to_fill)
                ]

                if len(compatible_indices) == 1:
                    df.loc[compatible_indices[0], 'Cabin'] = cabin

    return fill_deck_and_side_from_cabin(df)
                
                        

In [286]:
df = empty_room_one_compatible(df)



In [287]:
df_back = df.copy()


In [288]:
df = df_back.copy()

In [289]:
df.iloc[2414]

PassengerId                  1688_02
HomePlanet                     Earth
CryoSleep                       True
Cabin                        G/262/S
Destination              TRAPPIST-1e
Age                              7.0
VIP                            False
RoomService                      NaN
FoodCourt                        0.0
ShoppingMall                     0.0
Spa                              0.0
VRDeck                           0.0
Name               Claura Webstenson
Set                            Train
Transported                    False
Group                           1688
GroupNumber                        2
Deck                               G
Number                           262
Side                               S
FirstName                     Claura
LastName                  Webstenson
GroupSize                          3
Bills                            0.0
potential_decks                 None
potential_sides                 None
Name: 2414, dtype: object

In [290]:
df.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 41
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  41
Number                41
Side                  41
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12929
potential_sides    12929
dtype: int64

## No free rooms left so shares

In [291]:
def no_free_rooms_so_shares_3(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = []
        for deck in passenger.potential_decks:
            for side in passenger.potential_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number
                if len(top_room) > 0:
                    top_room_number_before = max(top_room)
                else:
                    continue

                smallest_room = after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number
                
                if len(smallest_room) > 0:
                    smallest_room_number_after = min(smallest_room)
                else:
                    continue
                if top_room_number_before + 1 < smallest_room_number_after:   
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
                        
        if len(options) == 0:
            if len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks)) & (df.Side.isin(passenger.potential_sides)) ].dropna(subset = 'Cabin').Cabin.unique()) == 1:
                df.loc[index,'Cabin'] = df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))& (df.Side.isin(passenger.potential_sides))].dropna(subset = 'Cabin').Cabin.iloc[0]
     

    
    return fill_deck_and_side_from_cabin(df)
    


In [292]:
df = no_free_rooms_so_shares_3(df)

In [293]:
df.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 41
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  41
Number                41
Side                  41
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12929
potential_sides    12929
dtype: int64

In [294]:
df.iloc[1251]

PassengerId             0884_02
HomePlanet                Earth
CryoSleep                 False
Cabin                   F/181/P
Destination         TRAPPIST-1e
Age                        28.0
VIP                       False
RoomService                 0.0
FoodCourt                   0.0
ShoppingMall              762.0
Spa                         0.0
VRDeck                      0.0
Name               Erine Rilley
Set                        Test
Transported                 NaN
Group                       884
GroupNumber                   2
Deck                          F
Number                      181
Side                          P
FirstName                 Erine
LastName                 Rilley
GroupSize                     2
Bills                     762.0
potential_decks            None
potential_sides            None
Name: 1251, dtype: object

# in a group of one and only one option

In [295]:
def solo_group_and_one_option(df):
    empty_cabins = rooms_to_fill(df) 
    
    for index,passenger in df[(df.GroupSize == 1) & (df.Cabin.isna())].iterrows():
        options = []
        for deck in empty_cabins.keys():
            if deck not in passenger.potential_decks:
                continue
            
            for side in empty_cabins[deck]:
                if side not in passenger.potential_sides:
                    continue
                for cabin in empty_cabins[deck][side]:
                    room_number = int(cabin.split("/")[1])
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    top_room = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number
                    if len(top_room) > 0:
                        top_room_number_before = max(top_room)
                    else:
                        continue
                    smallest_room = after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number
                    
                    if len(smallest_room) > 0:
                        smallest_room_number_after = min(smallest_room)
                    else:
                        continue
                    if top_room_number_before < room_number:
                        if smallest_room_number_after > room_number:
                            options.append([deck,side,top_room_number_before,smallest_room_number_after])
                    
                            
                            
        if len(options) == 1:
            if options[0][2] + 2 == options[0][3]:
                df.loc[index,'Cabin'] = options[0][0] + "/" + str(int(options[0][2]) + 1) + "/" + options[0][1]
               
    df = fill_deck_and_side_from_cabin(df)
    
    return df
                
                    
                    

In [296]:
df = solo_group_and_one_option(df)

In [297]:
df.iloc[1251]

PassengerId             0884_02
HomePlanet                Earth
CryoSleep                 False
Cabin                   F/181/P
Destination         TRAPPIST-1e
Age                        28.0
VIP                       False
RoomService                 0.0
FoodCourt                   0.0
ShoppingMall              762.0
Spa                         0.0
VRDeck                      0.0
Name               Erine Rilley
Set                        Test
Transported                 NaN
Group                       884
GroupNumber                   2
Deck                          F
Number                      181
Side                          P
FirstName                 Erine
LastName                 Rilley
GroupSize                     2
Bills                     762.0
potential_decks            None
potential_sides            None
Name: 1251, dtype: object

In [298]:
df.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 41
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  41
Number                41
Side                  41
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12929
potential_sides    12929
dtype: int64

# fuctions to help

In [222]:


df.loc[4233,'Cabin'] = 'B/98/P'
df.loc[4254,'Cabin'] = 'B/99/P'
df.loc[6493,'Cabin'] = 'E/300/S'
df.loc[6514,'Cabin'] = 'E/301/S'
df.loc[12892,'Cabin'] = 'F/1785/S' # maybe only one is from this room and the other is joined in the other room
df.loc[12893,'Cabin'] = 'F/1785/S'
df.loc[9265,'Cabin'] = 'F/1267/S'
df.loc[9267,'Cabin'] = 'F/1267/S'
df.loc[8413,'Cabin'] = 'A/57/P'
df.loc[1429,'Cabin'] = 'E/58/P'

df = fill_deck_and_side_from_cabin(df)


In [223]:
def all_cabin_options_for_each_row(df):
    count = 0
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print("\nindex", index)
        print("passenger",passenger.PassengerId)
        print("GroupSize", passenger.GroupSize)
        options = []
        for deck in passenger.potential_decks:
            for side in passenger.potential_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                if pd.isna(top_room_number_before) or pd.isna(smallest_room_number_after):
                        continue
                if top_room_number_before + 1 != smallest_room_number_after:
                    
                    if top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)
        print(passenger.potential_sides)
        if len(options) == 0:
            count += 1
    print(count)


In [224]:
def all_rows_for_each_cabin_option(df):
    empty_cabins = rooms_to_fill(df) 
    for deck in empty_cabins.keys():
        for side in empty_cabins[deck].keys():
            for cabin in empty_cabins[deck][side]:
                room_number = int(cabin.split("/")[1])
                indices_of_compatible_rows = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                    if side not in row.potential_sides:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    before = list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())
                    after = list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())
                    if len(before) > 0:
                        if max(before) < room_number:
                            if len(after) > 0:
                                if min(after) > room_number:
                                    indices_of_compatible_rows.append(index)
                print()
                print("cabin", cabin)
                print(indices_of_compatible_rows)

In [299]:
all_cabin_options_for_each_row(df)


index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12, 14], ['C', 'S', 12, 14]]
['P', 'S']

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12, 14], ['C', 'S', 12, 14]]
['P', 'S']

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19, 22]]
['P']

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19, 22]]
['P']

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19, 22]]
['P']

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57, 59]]
['P']

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39, 41], ['D', 'S', 35, 37], ['E', 'P', 57, 59]]
['P', 'S']

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39, 41], ['D', 'S', 35, 37]]
['P', 'S']

index 2442
passenger 1709_03
GroupSize 7
[]
['S']

index 2970
passenger 2092_03
GroupSize 5
[]
['S']

index 3529
passenger 2513_01
GroupSize 1
[['E', 'P', 149, 151], ['F', 'P', 518, 520]]
['P', 'S']

index 3530
passenger 2514_01
GroupSize 1
[['E', 'P', 149, 151], ['F', 'P', 518, 520]]
['P', 'S']

index 4233
passenger 3034_01
Group

In [228]:
df.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 31
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  31
Number                31
Side                  31
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12938
potential_sides    12938
dtype: int64

In [22]:
all_rows_for_each_cabin_option(df2)

NameError: name 'df2' is not defined